## Project 3: Model Training with scikit-learn

### **Objective:**

* #### Train a machine learning model to predict shipment times..

### **Methods Applied**:
1. Feature Engineering
2. Split dataset into training/testing sets
3. Normalize numerical features
4. Train regression models
5. Evaluate model performance

In [2]:
# Import necessary libraries
import pandas as pd                 # Data manipulation and analysis
import matplotlib.pyplot as plt     # Data visualization
import seaborn as sns               # Enhanced data visualization
import numpy as np                  # Numerical operations

# Scikit-learn components
from sklearn.model_selection import train_test_split  # Dataset splitting
from sklearn.linear_model import LinearRegression     # Linear regression model
from sklearn.ensemble import RandomForestRegressor    # Random Forest model
from sklearn.svm import SVR                           # Support Vector Regression
from sklearn.metrics import mean_squared_error, r2_score        # Evaluation metric
from sklearn.preprocessing import StandardScaler      # Feature scaling

# Gradient boosting libraries
from lightgbm import LGBMRegressor   # LightGBM implementation
from xgboost import XGBRegressor     # XGBoost implementation

### Data Loading
First, we load the preprocessed dataset containing shipment information.

## It is important to note that this data was sourced from Kaggle, however I have done the necessary preprocessing of the data in a separate notebook, save the cleaned data, and will now import it from my local machine for model training, evaluation and preprocessing.

In [4]:
# Load preprocessed shipment data
shipment_df = pd.read_csv('shipment_df.csv')

# Display first few rows to inspect data
shipment_df.head()

,routes,shipping_times,distance(km),number_of_products_sold,transportation_modes,location
0,Route B,3,1016,254,Air,Mumbai
1,Route C,4,967,510,Rail,Mumbai
2,Route A,3,1011,177,Air,Mumbai
3,Route A,4,1039,195,Rail,Chennai
4,Route B,7,1058,281,Sea,Kolkata


### Feature Engineering
We convert categorical variables into numerical representations using one-hot encoding. This transforms:
- transportation_modes
- location
- routes 
into binary columns that ML models can process.

In [6]:
# One-hot encode categorical features
shipment_df = pd.get_dummies(
    shipment_df, 
    columns=['transportation_modes', 'location', 'routes'], 
    drop_first=True  # Avoid dummy variable trap
).astype('int64')   # Convert boolean to integer

# Verify new features
print(f"New features: {list(shipment_df.columns)}")

New features: ['shipping_times', 'distance(km)', 'number_of_products_sold', 'transportation_modes_Rail', 'transportation_modes_Road', 'transportation_modes_Sea', 'location_Chennai', 'location_Delhi', 'location_Kolkata', 'location_Mumbai', 'routes_Route B', 'routes_Route C']


### Feature Preprocessing
We separate features from target variable and normalize numerical features to ensure equal contribution to models.

In [8]:
# Separate target variable (shipping times) from features
target = 'shipping_times'
X = shipment_df.drop(columns=[target])
y = shipment_df[target]

# Identify numerical columns for scaling
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Initialize and fit scaler to numerical features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Verify scaled features
print("Scaled feature summary:")
print(X.describe())

Scaled feature summary:
       distance(km)  number_of_products_sold  transportation_modes_Rail  \
count  2.500000e+04             2.500000e+04               2.500000e+04   
mean   3.950618e-17            -6.473044e-17               6.707523e-17   
std    1.000020e+00             1.000020e+00               1.000020e+00   
min   -1.476448e+00            -1.734455e+00              -5.775350e-01   
25%   -1.259257e+00            -8.657197e-01              -5.775350e-01   
50%    5.212785e-01             1.339916e-02              -5.775350e-01   
75%    6.632052e-01             8.648293e-01               1.731497e+00   
max    2.015810e+00             1.723182e+00               1.731497e+00   

       transportation_modes_Road  transportation_modes_Sea  location_Chennai  \
count               2.500000e+04              2.500000e+04      2.500000e+04   
mean                5.286438e-17             -7.588596e-17      9.663381e-18   
std                 1.000020e+00              1.000020e+00  

### Dataset Splitting
We split data into training (75%) and testing (25%) sets while maintaining consistent shuffling.

In [10]:
# Split data with fixed random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    random_state=30
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 18750
Testing samples: 6250


### Model Training
We train multiple regression models to compare performance:
1. Linear Regression (baseline)
2. Random Forest (ensemble)
3. Support Vector Regression
4. LightGBM (gradient boosting)
5. XGBoost (gradient boosting)

In [12]:
# Initialize models with hyperparameters
linear_model = LinearRegression()
forest_model = RandomForestRegressor(random_state=32)
svr_model = SVR(kernel='rbf', C=10, epsilon=0.1)
lgbm_model = LGBMRegressor(n_estimators=5, learning_rate=0.6, max_depth=7, random_state=32)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=32)

# Train models
linear_model.fit(X_train, y_train)
forest_model.fit(X_train, y_train)
svr_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002889 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 537
[LightGBM] [Info] Number of data points in the train set: 18750, number of used features: 11
[LightGBM] [Info] Start training from score 3.298827


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

### Model Evaluation
We evaluate performance using:
- Mean Squared Error (MSE): Lower is better
- R² Score: Higher is better (1 = perfect prediction)

In [14]:
# Generate predictions for each model
linear_pred = linear_model.predict(X_test)
forest_pred = forest_model.predict(X_test)
svr_pred = svr_model.predict(X_test)
lgbm_pred = lgbm_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

# Create prediction dictionary
model_predictions = {
    "Linear Regression": linear_pred,
    "Random Forest": forest_pred,
    "SVR": svr_pred,
    "LightGBM": lgbm_pred,
    "XGBoost": xgb_pred
}

# Import r2_score for consistent evaluation
from sklearn.metrics import r2_score

# Evaluate and compare all models
print("{:<20} {:<15} {:<10}".format('Model', 'MSE', 'R² Score'))
print("-" * 45)
for name, pred in model_predictions.items():
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    
    print("{:<20} {:<15.4f} {:<10.4f}".format(name, mse, r2))

Model                MSE             R² Score  
---------------------------------------------
Linear Regression    0.7589          0.7794    
Random Forest        0.2828          0.9178    
SVR                  0.3451          0.8997    
LightGBM             0.2533          0.9264    
XGBoost              0.2549          0.9259    


### Making Predictions
Let's make sample predictions using our trained models on new data

In [16]:
X.columns

Index(['distance(km)', 'number_of_products_sold', 'transportation_modes_Rail',
       'transportation_modes_Road', 'transportation_modes_Sea',
       'location_Chennai', 'location_Delhi', 'location_Kolkata',
       'location_Mumbai', 'routes_Route B', 'routes_Route C'],
      dtype='object')

In [18]:
# Generate predictions and evaluate models
models = {
    "Linear Regression": linear_model,
    "Random Forest": forest_model,
    "SVR": svr_model,
    "LightGBM": lgbm_model,
    "XGBoost": xgb_model
}

# Create new sample data (adapt values based on your features)
new_sample = pd.DataFrame({
    'distance(km)': [1016],
    'number_of_products_sold': [254],
    'transportation_modes_Rail': [0],
    'transportation_modes_Road': [0],
    'transportation_modes_Sea': [0],
    'location_Chennai': [0],
    'location_Delhi': [0],
    'location_Kolkata': [0],
    'location_Mumbai':[1],
    'routes_Route B': [1],
    'routes_Route C': [0]
})

# Preprocess new sample (same as training data)
new_sample_scaled = new_sample.copy()
new_sample_scaled[num_cols] = scaler.transform(new_sample_scaled[num_cols])

# Generate predictions
print("\nSample Prediction:")
print(f"Input features: \n{new_sample}\n")
for name, model in models.items():
    pred = model.predict(new_sample_scaled)
    print(f"{name}: {pred[0]:.2f} days")


Sample Prediction:
Input features: 
   distance(km)  number_of_products_sold  transportation_modes_Rail  \
0          1016                      254                          0   

   transportation_modes_Road  transportation_modes_Sea  location_Chennai  \
0                          0                         0                 0   

   location_Delhi  location_Kolkata  location_Mumbai  routes_Route B  \
0               0                 0                1               1   

   routes_Route C  
0               0  

Linear Regression: 3.00 days
Random Forest: 2.85 days
SVR: 2.09 days
LightGBM: 2.51 days
XGBoost: 2.50 days


### To predict multiple rows from existing data using indexes

In [21]:
# 1. Choose the row indices to predict (can be any list, slice, or mask)
rows_to_predict = [0, 2, 4]

# 2. Grab that subset from the original features DataFrame `X` usinf iloc
X_subset = X.iloc[rows_to_predict].copy()   # Extras brackets to keep it a DataFrame

# 3. Run your models in batch
print(f"Predicting for rows {rows_to_predict}:")
for name, model in models.items():
    preds = model.predict(X_subset)         # returns an array of length len(rows_to_predict)

    
    # Print each row’s prediction
    for idx, row_idx in enumerate(rows_to_predict):
        print(f" {name} prediction for row {row_idx}: {preds[idx]:.0f} days")
        print(f"Actual value for row {row_idx} is: {y[row_idx]:.0f} days")
        loss = preds[idx] - y[row_idx]
        print(f"Prediction loss = {loss:.0f} days\n")
        print("-" * 65)


Predicting for rows [0, 2, 4]:
 Linear Regression prediction for row 0: 3 days
Actual value for row 0 is: 3 days
Prediction loss = 0 days

-----------------------------------------------------------------
 Linear Regression prediction for row 2: 3 days
Actual value for row 2 is: 3 days
Prediction loss = -0 days

-----------------------------------------------------------------
 Linear Regression prediction for row 4: 6 days
Actual value for row 4 is: 7 days
Prediction loss = -1 days

-----------------------------------------------------------------
 Random Forest prediction for row 0: 3 days
Actual value for row 0 is: 3 days
Prediction loss = -0 days

-----------------------------------------------------------------
 Random Forest prediction for row 2: 2 days
Actual value for row 2 is: 3 days
Prediction loss = -1 days

-----------------------------------------------------------------
 Random Forest prediction for row 4: 7 days
Actual value for row 4 is: 7 days
Prediction loss = -0 days

### On these three specific rows the plain Linear Regression “wins” (it only misses row 4 by 1 day, just like LightGBM and Random Forest, whereas XGBoost misses twice and SVR once). But those three points are just a tiny slice of the test set. Here’s why it doesn't mean that Linear Regression is actually better:

#### Overall metrics vs. individual points

1. The test‐set MSE and R² summarize performance over all held-out examples.

* Linear Regression:

    * MSE = 0.7589

    * R² ≈ 0.7794

* LightGBM (best performer):

    * MSE = 0.2533

    * R² ≈ 0.9264

Those numbers tell us that, on average, LightGBM’s predictions are much closer to the true values, and it explains about 92.6% of the variance versus only ~78% for the linear model.

2. Small-sample noise

* With only three rows, random fluctuations can make a weaker model look better on that tiny subset. It’s like flipping a biased coin three times and getting the “rare” result every time—you’d be misled about which coin is really fairer.

3. Robustness and generalization

* The ensemble methods not only have lower average squared error, they’re also capturing non-linearities and interactions that your simple line can’t. Even if those extra complexities didn’t show up in rows 0, 2, and 4, they’ll pay dividends across the rest of your data.

### Conclusion
**Key Findings**:
- Gradient boosting models (LightGBM and XGBoost) showed superior performance
- Linear Regression had highest error, suggesting non-linear relationships
- Feature scaling and encoding were crucial for model performance

**Recommendations**:
1. Explore feature importance analysis
2. Perform hyperparameter tuning
3. Investigate feature interactions
4. Consider time-based features
5. Test neural net
6. Evaluate on more examples (or k-fold CV) to see that the tree-based models consistently outperform linear.
7. Plot the residuals (prediction − actual) across all test points to check for any systematic biases.
8. If interpretability is important, I can still keep the Linear model as a quick baseline, but for pure predictive power, LightGBM or Random Forest are the winners here.work models